# Using Model Endpoint

## Requirements

- Authenticated to gcloud (```gcloud auth application-default login```)

This notebook demonstrate how we can leverage model endpoint for integrating with external service. In the previous examples, we deployed models to create a version endpoints. The version endpoint is specific to a certain model version and the url will change everytime a new version is deployed. Hence, version endpoint is not suitable to be shared publicly or being used for integration. 

For publicly sharing your model URL it is recommended to use model endpoint. Model endpoint is a model URL that is not specific to certain version. The benefit of using model endpoint is that we can specify which version of the model being used (by redirecting traffic from one model version to another) without having our client to change the url.

In [ ]:
!pip install --upgrade -r requirements.txt > /dev/null

In [ ]:
import merlin
import warnings
import os
from sklearn import svm
from sklearn import datasets
from joblib import dump
from sklearn.datasets import load_iris
from merlin.model import ModelType
warnings.filterwarnings('ignore')

## 1. Initialize MLP Resources

### 1.1 Set MLP Server

In [ ]:
# Set MLP Server
merlin.set_url("localhost:8080/api/merlin")

### 1.2 Set Active Project

`project` represent a project in real life. You may have multiple model within a project.

`merlin.set_project(<project_name>)` will set the active project into the name matched by argument. You can only set it to an existing project. If you would like to create a new project, please do so from the MLP console at http://localhost:8080/projects/create.

In [ ]:
merlin.set_project("sample")

### 1.3 Set Active Model

`model` represents an abstract ML model. Conceptually, `model` in MLP is similar to a class in programming language. To instantiate a `model` you'll have to create a `model_version`.

Each `model` has a type, currently model type supported by MLP are: sklearn, xgboost, tensorflow, pytorch, and user defined model (i.e. pyfunc model).

`model_version` represents a snapshot of particular `model` iteration. You'll be able to attach information such as metrics and tag to a given `model_version` as well as deploy it as a model service.

`merlin.set_model(<model_name>, <model_type>)` will set the active model to the name given by parameter, if the model with given name is not found, a new model will be created.

In [ ]:
merlin.set_model("sklearn-sample", ModelType.SKLEARN)

## 2. Train and Deploy

### 2.1 Train and Upload Model

`merlin.new_model_version()` is a convenient method to create a model version and start its development process. It is equal to following codes:

```
v = model.new_model_version()
v.start()
v.log_model(model_dir=model_dir)
v.finish()
```

In [ ]:
model_dir = "sklearn-model"
MODEL_FILE = "model.joblib"

url = ""

# Create new version of the model
with merlin.new_model_version() as v:
    clf = svm.SVC(gamma='scale')
    iris = datasets.load_iris()
    X, y = iris.data, iris.target
    clf.fit(X, y)
    dump(clf, os.path.join(model_dir, MODEL_FILE))
    
    # Upload the serialized model to MLP
    merlin.log_model(model_dir=model_dir)

## 2.2 Deploy Model

In [ ]:
endpoint = merlin.deploy(v)

Once deployed you will receive a version endpoint. Notice that the url has following format: sklean-sample-http://<model_version>.<domain>/v1/models/sklearn-sample-<model_version> . Everytime a new version is created, the <model_version> part will also change.  

## 2.3 Send Test Request

In [ ]:
%%bash -s "$endpoint.url"
curl -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

## 2.4 Create Model Endpoint

To create a model endpoint we can use `serve_traffic` method passing in a dictionary containing the version endpoint which will serve the traffic and the percentage of the traffic. In this case, we are going to use the version endpoint that we deployed before with 100% of traffic.

In [ ]:
model_endpoint = merlin.serve_traffic({endpoint:100})

In [ ]:
model_endpoint

Sending request to the model endpoint will return the same result as the version endpoint

In [ ]:
%%bash -s "$model_endpoint.url"
curl -X POST $1 -d '{
  "instances": [
    [2.8,  1.0,  6.8,  0.4],
    [3.1,  1.4,  4.5,  1.6]
  ]
}'

## 2.5 Undeploy Model Endpoint

To undeploy model endpoint we can use `stop_serving_traffic` method passing in an environment name where the model endpoint is. By undeploying model endpoint you stop serving traffic to the model endpoint URL. The version endpoint target is still running.

In [ ]:
merlin.stop_serving_traffic(endpoint.environment_name)